# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

# Access maps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read csv data 
city_weather_df = pd.read_csv("../Output/cities.csv")
# display output
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,39.20,81,0,5.75,CL,1619329078
1,East London,-33.0153,27.9116,59.00,67,0,9.22,ZA,1619328812
2,Hobart,-42.8794,147.3294,60.80,44,75,20.71,AU,1619328981
3,Kapaa,22.0752,-159.3190,75.20,73,40,16.11,US,1619329084
4,Tuktoyaktuk,69.4541,-133.0374,17.60,93,20,10.36,CA,1619329084
...,...,...,...,...,...,...,...,...,...
550,Uddevalla,58.3478,11.9424,37.00,64,0,12.66,SE,1619329251
551,Evensk,61.9500,159.2333,29.35,91,100,10.63,RU,1619329252
552,Bandar-e Lengeh,26.5579,54.8807,86.00,51,70,6.91,IR,1619329051
553,Kaeo,-35.1000,173.7833,62.01,63,71,5.01,NZ,1619329252


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Set the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

# Setup Heatmap layer variable
humidity = city_weather_df["Humidity"]

In [4]:
# Plot gmaps
map_fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Add heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, point_radius=3,max_intensity=max(humidity))

# Display gmap with heat layer
map_fig.add_layer(heat_layer)
map_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down DataFrame to ideal weather condition
ideal_weather_df = city_weather_df.loc[(city_weather_df["Wind Speed"] < 10) & (city_weather_df["Cloudiness"]== 0) &
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
112,Atar,20.5169,-13.0499,71.17,33,0,8.14,MR,1619329118
155,Seoul,37.5683,126.9778,71.60,33,0,6.91,KR,1619328879
187,Caravelas,-17.7125,-39.2481,72.46,91,0,4.34,BR,1619329142
519,Palestine,31.9216,35.2033,70.00,77,0,1.14,PS,1619329242
524,Pitimbu,-7.4706,-34.8086,73.40,100,0,4.61,BR,1619329244


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = ideal_weather_df.loc[:,["City","Country","Lat","Lng"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
112,Atar,MR,20.5169,-13.0499,
155,Seoul,KR,37.5683,126.9778,
187,Caravelas,BR,-17.7125,-39.2481,
519,Palestine,PS,31.9216,35.2033,
524,Pitimbu,BR,-7.4706,-34.8086,


In [7]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
# Iterrate through hotel_df to set parameters and collect hotel data
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Save hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 112: Atar.
Closest hotel in Atar is Bab Sahara.
--------------------Ending Search
Retrieving Results for Index 155: Seoul.
Closest hotel in Seoul is Lotte Hotel Seoul.
--------------------Ending Search
Retrieving Results for Index 187: Caravelas.
Closest hotel in Caravelas is Mar Doce de Abrolhos.
--------------------Ending Search
Retrieving Results for Index 519: Palestine.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 524: Pitimbu.
Closest hotel in Pitimbu is Asenza Beach Resort.
--------------------Ending Search


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
# Add markers and info_box_content to show when user clicks it
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
map_fig.add_layer(markers)

# Display figure
map_fig

Figure(layout=FigureLayout(height='420px'))